# 🧹 **Data Cleaning & Feature Engineering** — Housing Prices in India

---

### 🔗 **Notebook Context**
This notebook is the **second stage** of the *Housing Prices in India* project.  
In the previous notebook, we performed an **Exploratory Data Analysis (EDA)** to understand data distributions, relationships, and patterns.  

Here, we’ll implement the **data cleaning and feature engineering** processes guided by insights from the EDA.  
The cleaned dataset will serve as the foundation for the machine learning model in the next stage.

---

## 🧭 **Objectives**

1. Load and review the raw dataset  
2. Implement data cleaning strategies identified during EDA  
3. Handle missing values, duplicates, and inconsistent entries  
4. Engineer meaningful features for improved model performance  
5. Save the cleaned and transformed dataset for modeling  

---

## 📦 **1. Setup & Data Loading**
- Import relevant libraries  
- Load dataset  
- Verify data integrity (shape, columns, dtypes)  

In [26]:
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from IPython.display import display

In [15]:
df = pd.read_csv('../data/train.csv')
display(df.head())
display(df.info())
display(df.shape)

POSTED_BY  UNDER_CONSTRUCTION  RERA  BHK_NO. BHK_OR_RK    SQUARE_FT  \
0     Owner                   0     0        2       BHK  1300.236407   
1    Dealer                   0     0        2       BHK  1275.000000   
2     Owner                   0     0        2       BHK   933.159722   
3     Owner                   0     1        2       BHK   929.921143   
4    Dealer                   1     0        2       BHK   999.009247   

   READY_TO_MOVE  RESALE                      ADDRESS  LONGITUDE   LATITUDE  \
0              1       1        Ksfc Layout,Bangalore  12.969910  77.597960   
1              1       1    Vishweshwara Nagar,Mysore  12.274538  76.644605   
2              1       1             Jigani,Bangalore  12.778033  77.632191   
3              1       1  Sector-1 Vaishali,Ghaziabad  28.642300  77.344500   
4              0       1             New Town,Kolkata  22.592200  88.484911   

   TARGET(PRICE_IN_LACS)  
0                   55.0  
1                   51.0  
2                   43.0  
3                   62.5  
4                   60.5

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29451 entries, 0 to 29450
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   POSTED_BY              29451 non-null  object 
 1   UNDER_CONSTRUCTION     29451 non-null  int64  
 2   RERA                   29451 non-null  int64  
 3   BHK_NO.                29451 non-null  int64  
 4   BHK_OR_RK              29451 non-null  object 
 5   SQUARE_FT              29451 non-null  float64
 6   READY_TO_MOVE          29451 non-null  int64  
 7   RESALE                 29451 non-null  int64  
 8   ADDRESS                29451 non-null  object 
 9   LONGITUDE              29451 non-null  float64
 10  LATITUDE               29451 non-null  float64
 11  TARGET(PRICE_IN_LACS)  29451 non-null  float64
dtypes: float64(4), int64(5), object(3)
memory usage: 2.7+ MB


None

(29451, 12)

Now, the dataset remains in the right format as expected.
## Data Cleaning
### **Rename column names**
The first step is to rename the column, set the names to lower case, then, set all binary values to their appropriate values so that we can use Label encoding on them.

In [16]:
# Rename columns to lower case
try:
    df.rename(columns={'TARGET(PRICE_IN_LACS)': 'PRICE'})
    df.rename(columns=lambda x: x.lower(), inplace=True)

    df.rename(columns={
            'rera': 'rera_approved',
            'bhk_no.': 'num_of_rooms',
            'square_ft': 'total_area',
            'target(price_in_lacs)': 'price'
        }, inplace=True)
except Exception as e:
    print(f"Error renaming columns: {e}")

In [17]:
pct_duplicated = 100 * df.duplicated().sum() / len(df)
print(f"Percentage of duplicated rows: {pct_duplicated:.2f}%")

Percentage of duplicated rows: 1.36%


### **Handle Duplicates**

Now, the next step is to drop duplicate data. Though, this is allowed it is allowed in the real-estate industry that we can have multiple, identical houses in the same compound sharing exactyly the same attributes including prices. 

In our case, the house numbers or block numbers are not provided, so duplicates are allowed, for machine learning modelling, only one is enough to provide the required information.

In addition, since only **1.36%** of the data has duplicates, we can drop the duplicates.

In [18]:
df.drop_duplicates(inplace=True)
assert df.duplicated().sum() == 0

### **Handle Binary Features**
The binary features which are already encoded as integers need to be set to their semantic meaning so that Label Encoder can appropriately work on them during data preparation for modelling.



In [19]:
df['rera_approved'] = df['rera_approved'].map({1: 'Yes', 0: 'No'})
df['ready_to_move'] = df['ready_to_move'].map({1: 'Yes', 0: 'No'})
df['resale'] = df['resale'].map({1: 'Yes', 0: 'No'})
df['under_construction'] = df['under_construction'].map({1: 'Yes', 0: 'No'})

In [20]:
df.head()

posted_by under_construction rera_approved  num_of_rooms bhk_or_rk  \
0     Owner                 No            No             2       BHK   
1    Dealer                 No            No             2       BHK   
2     Owner                 No            No             2       BHK   
3     Owner                 No           Yes             2       BHK   
4    Dealer                Yes            No             2       BHK   

    total_area ready_to_move resale                      address  longitude  \
0  1300.236407           Yes    Yes        Ksfc Layout,Bangalore  12.969910   
1  1275.000000           Yes    Yes    Vishweshwara Nagar,Mysore  12.274538   
2   933.159722           Yes    Yes             Jigani,Bangalore  12.778033   
3   929.921143           Yes    Yes  Sector-1 Vaishali,Ghaziabad  28.642300   
4   999.009247            No    Yes             New Town,Kolkata  22.592200   

    latitude  price  
0  77.597960   55.0  
1  76.644605   51.0  
2  77.632191   43.0  
3  77.344500   62.5  
4  88.484911   60.5

### **Handle Outliers**
In the real-estate industry, we need to know precisely what data point does not make sense, especially for numerical columns.

In [28]:
df[['longitude', 'latitude', 'num_of_rooms', 'total_area', 'price']]\
    .plot(kind='box', subplots=True, layout=(1, 5), figsize=(20, 5))
plt.suptitle('Box plots for Numerical Features')
plt.show()